In [13]:
import chess
import chess.pgn
import io
import tensorflow as tf
import numpy as np

In [14]:


class ChessConverter:
    @staticmethod
    def boardtofen(fen):
        binary = ''
        for element in fen:
            if(element == "P"):
                binary += "0100000"
            elif(element == "R"):
                binary += "0010000"
            elif(element == "N"):
                binary += "0001000"
            elif(element == "B"):
                binary += "0000100"
            elif(element == "Q"):
                binary += "0000010"
            elif(element == "K"):
                binary += "0000001"
            elif(element == "p"):
                binary += "1100000"
            elif(element == "r"):
                binary += "1010000"
            elif(element == "n"):
                binary += "1001000"
            elif(element == "b"):
                binary += "1000100"
            elif(element == "q"):
                binary += "1000010"
            elif(element == "k"):
                binary += "1000001"

            elif(element == "1"):
                binary += "0000000"
            elif(element == "2"):
                binary += "0000000"
                binary += "0000000"
                
            elif(element == "3"):
                binary += "0000000"
                binary += "0000000"
                binary += "0000000"
            elif(element == "4"):
                binary += "0000000"
                binary += "0000000"
                binary += "0000000"
                binary += "0000000"
            elif(element == "5"):
                binary += "0000000"
                binary += "0000000"
                binary += "0000000"
                binary += "0000000"
                binary += "0000000"
            elif(element == "6"):
                binary += "0000000"
                binary += "0000000"
                binary += "0000000"
                binary += "0000000"
                binary += "0000000"
                binary += "0000000"
                
            elif(element == "7"):
                binary += "0000000"
                binary += "0000000"
                binary += "0000000"
                binary += "0000000"
                binary += "0000000"
                binary += "0000000"
                binary += "0000000"
            elif(element == "8"):
                binary += "0000000"
                binary += "0000000"
                binary += "0000000"
                binary += "0000000"
                binary += "0000000"
                binary += "0000000"
                binary += "0000000"
                binary += "0000000"
            elif(element == "w"):
                binary += "0"
            elif(element == "b"):
                binary += "1"

        return binary[:449]
    
    def result_to_binary(self, result):
        if(result == "1-0"):
            return 1
        elif(result == "0-1"):
            return -1
        else:
            return 0
    





            

def generate_fen(game):
    board = chess.Board()
    moves = game.mainline_moves()
    fen_list = []

    for move in moves:
        board.push(move)
        fen_list.append(board.fen())

    return fen_list


def process_game(game):
    # Process each game here
    # You can access game metadata and moves within this function
    
    pgn = chess.pgn.read_game(io.StringIO(pgn_text))
    fen_list = generate_fen(pgn)

    converter = ChessConverter()
    

    for i, fen in enumerate(fen_list):
        print(f"Position {i + 1}: {converter.boardtofen(fen)} ")


In [12]:
pgn_file = open("lichess_db_standard_rated_2017-02.pgn")

# Specify the interval to start processing games
starting_line_interval = 18

# Variable to keep track of the current line number
current_line = 1

Fens = []
Results = []

while True:
    # Check if the current line is within the desired interval
    print(f"\rProgress: {(current_line/(183552715 * .04))*100}%")

    if (current_line + 1) % starting_line_interval == 0:
        game = chess.pgn.read_game(pgn_file)

        if game is None:
            break
        if current_line == (183552715 * .04):
            break

        # Process each game here
        # You can access game metadata and moves within this loop
        fen_list = generate_fen(game)
        result = game.headers["Result"]
        converter = ChessConverter()
        binaryResult = converter.result_to_binary(result)

        for i, fen in enumerate(fen_list):
            Fens.append(np.array([int(j) for j in (converter.boardtofen(fen))]))
            Results.append(binaryResult)

    # Read the next line
    line = pgn_file.readline()

    if not line:
        break

    current_line += 1

pgn_file.close()

Progress: 1.362006549453654e-05%
Progress: 2.724013098907308e-05%
Progress: 4.086019648360963e-05%
Progress: 5.448026197814616e-05%
Progress: 6.810032747268271e-05%
Progress: 8.172039296721925e-05%
Progress: 9.534045846175579e-05%
Progress: 0.00010896052395629232%
Progress: 0.00012258058945082887%
Progress: 0.00013620065494536541%
Progress: 0.00014982072043990196%
Progress: 0.0001634407859344385%
Progress: 0.00017706085142897503%
Progress: 0.00019068091692351158%
Progress: 0.00020430098241804815%
Progress: 0.00021792104791258464%
Progress: 0.0002315411134071212%


NameError: name 'chess' is not defined

In [15]:
model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(449),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1),
])


In [ ]:
model.compile(
    optimizer='adam',
    loss= "mean_squared_error")

model.fit(
    x = np.asarray(Fens),
    y = np.asarray(Results),
    epochs = 5,
    batch_size = 100,
    validation_split = 0.2
)

model.save("predictor.h5")